In [ ]:
import cv2
import numpy as np
import h5py
from imgaug import augmenters  as iaa


In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the h5 file into memory because it's small enough

data = []
f = h5py.File("drive/MyDrive/datasets/gtav/balanced_data.h5")
images = f["images"][:]
images = np.array(images).reshape(-1, 160, 120, 1)
feedbacks = f["feedbacks"][:]

for i in range(f["images"].shape[0]):
  data.append([images[i], feedbacks[i]])

print(len(data))

In [ ]:
# Double the dataset size by flipping the image and steering

for i in range(len(images)):
    image = cv2.flip(images[i], 1)
    feedback = feedbacks[i]
    data.append([image, [-feedback[0], feedback[1]]])

print(len(data))

In [ ]:
# Split data into train and val sets

np.random.shuffle(data)

border = round(len(data) * 0.8)

train = np.asarray(data[:border], dtype=object)
val = np.asarray(data[border:], dtype=object)

print(train.shape)
print(val.shape)

In [ ]:
#Image augmentation used to generalize the performance of the model
def zoom(image):
  zoom_img = iaa.Affine(scale = (1, 1.3))
  image = zoom_img.augment_image(image)
  return image

def pan(image):
  pan = iaa.Affine(translate_percent={'x': (-0.1, 0.1), 'y': (-0.1, 0.1)})
  image = pan.augment_image(image)
  return image

def img_random_brightness(image):
  brightness = iaa.Multiply((0.2, 1.2))
  image = brightness.augment_image(image)
  return image

def img_random_flip(image, choice):
  image = cv2.flip(image, 1)
  steering=choice[0]
  throttle=choice[1]
  steering = -steering
  new_choice = [steering, throttle]
  return image, new_choice

def random_augment(image, choice):
    #image = npimg.imread(image)
    #return image, choice
    if np.random.rand() < 0.5:
      image = pan(image)
    if np.random.rand() < 0.5:
      image = zoom(image)
    if np.random.rand() < 0.5:
      image = img_random_brightness(image)
    if np.random.rand() < 0.5:
      image, choice = img_random_flip(image, choice)
    
    
    return image, choice

In [ ]:
def data_generator(data, batch_size, is_train):
  while True:
    images = []
    feedbacks = []

    samples = data[np.random.choice(data.shape[0], size=batch_size, replace=False), :]
    for sample in samples:
      im, f = sample
      if is_train:
        im, f = random_augment(im, f)
      
      im = im.reshape(160 , 120, 1)
      images.append(im)
      feedbacks.append(f)
    yield np.array(images), np.array(feedbacks)

In [ ]:
from tensorflow.keras.layers import Conv2D, Dropout, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


model = Sequential()

model.add(Conv2D(24, (5, 5), activation='elu', strides=(2, 2), input_shape=(160, 120, 1)))
model.add(Conv2D(36, (5, 5), activation='elu', strides=(2, 2)))
model.add(Conv2D(48, (5, 5), activation='elu', strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='elu'))
model.add(Conv2D(64, (3, 3), activation='elu'))

model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(2))

model.compile(loss='mean_squared_error', metrics=["accuracy"],  optimizer=Adam(learning_rate=0.001))

In [ ]:
print(model.summary())

In [ ]:
epochs = 25

model.fit(x=data_generator(train, 300, True),
              #steps_per_epoch = 150,
              #validation_steps = 50,
              validation_data=data_generator(val, 100, False),
              steps_per_epoch=len(train) // 300,
              validation_steps=len(val) // 100,
              epochs=epochs)

In [ ]:
model.save('model.h5') 

In [ ]:
from google.colab import files
files.download('model.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>